<a href="https://colab.research.google.com/github/Krishgrv/sample-code/blob/master/QAProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install allennlp


     |████████████████████████████████| 7.6MB 2.8MB/s 
     |████████████████████████████████| 225kB 52.6MB/s 
     |████████████████████████████████| 61kB 25.1MB/s 
     |████████████████████████████████| 245kB 46.4MB/s 
     |████████████████████████████████| 163kB 39.5MB/s 
     |████████████████████████████████| 256kB 42.4MB/s 
     |████████████████████████████████| 51kB 15.8MB/s 
     |████████████████████████████████| 133kB 45.2MB/s 
     |████████████████████████████████| 655kB 40.8MB/s 
     |████████████████████████████████| 1.0MB 28.3MB/s 
  Created wheel for ftfy: filename=ftfy-5.6-cp36-none-any.whl size=44553 sha256=f8094f68ce4851d0122f3d30d4c3a4a1d07d5e0ae7b9fb988ca06b93658d263f
  Stored in directory: /root/.cache/pip/wheels/43/34/ce/cbb38d71543c408de56f3c5e26ce8ba495a0fa5a28eaaf1046
  Created wheel for numpydoc: filename=numpydoc-0.9.1-cp36-none-any.whl size=31872 sha256=8d26ee04c8928711125f76b5366421815039562edc3c3b58649440aab3496e13
  Stored in directory: /root/.cache/

In [0]:
from allennlp.predictors.predictor import Predictor
#predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")

100%|██████████| 46175392/46175392 [00:00<00:00, 58045205.11B/s]
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [27]:
import re
import sys
from allennlp.predictors.predictor import Predictor
from difflib import get_close_matches

def ReadInputFromFile(filepath):
   with open(filepath) as fp:
        global inputText
        global questions
        global answers
        global lqns
        global lans
        
        line = fp.readline()
        cnt = 1
        while line:
             #print("Line {}: {}".format(cnt, line.strip()))
             if (cnt == 1):
                inputText = line
             if (cnt >=2 and cnt <= 6):
                questions += line.rstrip('\r\n')
             if (cnt > 6):
                answers += line
             line = fp.readline()
             cnt += 1

        lqns = questions.split('?')
        lans = answers.split(';')


def LoadPredictor():
    global predictor
    predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")

def ProcessInputAndAnswerQuestions():
   global lqns
   global predictor
   global inputText

   qcnt = 1
   for qn in lqns:
    if (len(qn.strip()) != 0):
       print ('========')
       print ("{}. {}".format(qcnt,  qn + '?'))
       print('.........')
       result = predictor.predict(
                 passage=inputText,
                 question=qn
       )
       bestans = result['best_span_str']
       #print (bestans)
       lbestans = get_close_matches(bestans, lans)       
       if (len(lbestans) > 0):
           print ("Ans: {}".format(lbestans[0]))
       else:
           print ("Error: {}".format("Either Invalid question OR No appropriate answer provided!!"))
       qcnt += 1


#if (len(sys.argv) < 2):
#   print ("Usage : python readFile.py <Sample input file name>")
#   sys.exit(1)

#print ("Input file : ", sys.argv[1] )

#filepath = sys.argv[1]
filepath = "/tmp/sampleInput"
inputText = ""
questions = ""
answers = ""

ReadInputFromFile(filepath)
LoadPredictor()
ProcessInputAndAnswerQuestions()





/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:56: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


1. Which Zebras are endangered?
.........
Ans:  Grévy's zebra and the mountain zebra

2. What is the aim of the Quagga Project?
.........
Ans: aims to breed zebras that are phenotypically similar to the quagga
3. Which animals are some of their closest relatives?
.........
Ans: horses and donkeys
4. Which are the three species of zebras?
.........
Ans:  the plains zebra, the Grévy's zebra and the mountain zebra
5. Which subgenus do the plains zebra and the mountain zebra belong to?
.........
Ans: subgenus Hippotigris
